In [2]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/Mumbai_house_rent_99acers/Mumbai_99acers_cleaned.csv')
df

,society,type,location,bedrooms,built-up area,furnishing,age,floor,total floors,monthly rent
0,other society,Residential Apartment,Bandra (West),2,800.0,Furnished,5 to 10 years old,2,6,85000
1,other society,Residential Apartment,Khar West,2,1050.0,Furnished,1 to 5 years old,8,12,110000
2,Gajra Bhoomi Symphony,Residential Apartment,Sector-20 Koparkhairane,2,1050.0,Unfurnished,1 to 5 years old,14,17,32000
3,Crescent Bay,Residential Apartment,Parel,3,2275.0,Semifurnished,1 to 5 years old,40,41,150000
4,Ashwini CHS,Residential Apartment,Tilak Nagar,1,550.0,Semifurnished,5 to 10 years old,4,7,27000
...,...,...,...,...,...,...,...,...,...,...
12792,other society,Residential Apartment,Prabhadevi,3,2529.0,Furnished,1 to 5 years old,12,32,175000
12793,Runwal Greens,Residential Apartment,Mulund (West),3,1382.0,Semifurnished,0 to 1 years old,27,40,48000
12794,other society,Residential Apartment,other location,3,1750.0,Unfurnished,10+ years old,9,16,200000
12795,other society,Residential Apartment,Sector 19 Kharghar,2,950.0,Semifurnished,5 to 10 years old,8,10,25500


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12797 entries, 0 to 12796
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   society        12797 non-null  object 
 1   type           12797 non-null  object 
 2   location       12797 non-null  object 
 3   bedrooms       12797 non-null  int64  
 4   built-up area  12797 non-null  float64
 5   furnishing     12797 non-null  object 
 6   age            12797 non-null  object 
 7   floor          12797 non-null  int64  
 8   total floors   12797 non-null  int64  
 9   monthly rent   12797 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 999.9+ KB


# Machine Learning Model Building.
<a href = '#top'>Back on Top</a>

## OneHotEncoding Text features

In [19]:
temp = pd.get_dummies(df, drop_first=True)
corr = temp.corr()['monthly rent'].sort_values()

In [20]:
corr.head(10)


furnishing_Unfurnished                    -0.361680
location_Dombivli (East)                  -0.175266
society_Lodha Lakeshore Greens            -0.143397
location_other location                   -0.125308
location_Kharghar                         -0.108375
location_Sector 17 Ulwe                   -0.102996
location_Kasar Vadavali                   -0.094774
society_Chatrapati Shivaji Raje Complex   -0.093411
location_Ekta Nagar                       -0.093411
society_other society                     -0.093317
Name: monthly rent, dtype: float64

In [21]:
corr.tail(10)

society_Oberoi Esquire      0.172564
furnishing_Semifurnished    0.191221
location_Khar West          0.203989
society_Crescent Bay        0.238524
location_Parel              0.240461
floor                       0.379980
total floors                0.383646
bedrooms                    0.553519
built-up area               0.615666
monthly rent                1.000000
Name: monthly rent, dtype: float64

In [22]:
temp

,bedrooms,built-up area,floor,total floors,monthly rent,society_Acme Ozone,society_Adityavardhan Apartment,society_Ajmera Aeon,society_Ajmera Bhakti Park Sector 1 and 2,society_Ajmera Zeon,society_Alica Nagar CHS,society_Ashford Royale,society_Ashwini CHS,society_Bhagwati Bay Bliss,society_Bholenath Chembur Castle,society_Bhoomi Park,society_Carter Road,society_Central Avenue Apartments,society_Chaitanya Towers,society_Chatrapati Shivaji Raje Complex,society_Check,society_Co-oprative Society,society_Cosmos Jewels,society_Cosmos Lounge,society_Crescent Bay,society_DB Woods,society_Description :,society_Dheeraj Platinum Apartment,society_Diamond Garden,society_Divine Varsha,society_Dosti Acres,society_Dosti Ambrosia,society_Dosti Estates,society_Dosti Imperia,society_Dynamix Parkwoods,society_Emgee Greens,society_Evening Glory CHS,society_Evershine Millennium Paradise,society_Exquisit Falt,society_Exquisite Falt,...,location_Sector-35G Kharghar,location_Sector-50 Seawoods,location_Sector-58 Seawoods,location_Sector-6 Kharghar,location_Sector-6 Nerul,location_Sector-8 Charkop,location_Sector-8 Kharghar,location_Sector-8A Airoli,location_Sector-9 Ulwe,location_Sewri,location_Shastri Nagar,location_Sindhi Society Chembur,location_Tardeo,location_Teen Haath Naka,location_Thakur Complex,location_Thakur Village,location_Thane West,location_Tilak Nagar,location_Ulwe,location_Vakola,location_Vasant Vihar,location_Versova,location_Vijay Nagari,location_Vile Parle (East),location_Vile Parle (West),location_Virar West,location_Wadala,location_Wadala East,location_Waghbil,location_Worli,location_Worli Seaface,location_Yogidham,location_other location,furnishing_Not Mentioned,furnishing_Semifurnished,furnishing_Unfurnished,age_1 to 5 years old,age_10+ years old,age_5 to 10 years old,age_Not Mentioned
0,2,800.0,2,6,85000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2,1050.0,8,12,110000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,2,1050.0,14,17,32000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
3,3,2275.0,40,41,150000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
4,1,550.0,4,7,27000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12792,3,2529.0,12,32,175000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
12793,3,1382.0,27,40,48000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
12794,3,1750.0,9,16,200000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0
12795,2,950.0,8,10,25500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


## Preparing Training and Testing Datasets

In [23]:
x = temp.drop('monthly rent', axis = 1)
y = temp['monthly rent']
x.shape,y.shape

((12797, 355), (12797,))

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10237, 355), (2560, 355), (10237,), (2560,))

## Scaling Datasets to convert them into same range.

In [47]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
x_train = scaler.transform(X_train)
x_test = scaler.transform(X_test)

## Time to Built a ML model now 

## LinearRegression model with standard scaling.

In [48]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

linear_model = LinearRegression()
linear_model.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [51]:
r2_score(y_test, linear_model.predict(x_test))

0.8850250933959646

## Lets store it in a DataFrame. so that we can compare different model's scores in last.

In [52]:
Final_Scores = pd.DataFrame( index = ['With Training Data', 'With Test Data'])


In [53]:
Final_Scores['Linear Regression %'] = [r2_score(y_train, linear_model.predict(x_train))*100, 
                                       r2_score(y_test, linear_model.predict(x_test))*100]

##Ridge

In [54]:
ridge = Ridge()
ridge.fit(x_train,y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [55]:
r2_score(y_test, ridge.predict(x_test))

0.8850479799093492

In [56]:
Final_Scores['Ridge Regression %'] = [r2_score(y_train, ridge.predict(x_train))*100, 
                                       r2_score(y_test, ridge.predict(x_test))*100]

##Ridge

In [57]:
lasso = Lasso()
lasso.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 35058627569.295166, tolerance: 1333801373.1923687
  positive)


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [58]:
r2_score(y_test, lasso.predict(x_test))

0.8851027112279908

In [59]:
Final_Scores['Lasso Regression %'] = [r2_score(y_train, lasso.predict(x_train))*100, 
                                       r2_score(y_test, lasso.predict(x_test))*100]

##LabelEncoder

In [61]:
df1 = df.copy()
df1.head(3)

,society,type,location,bedrooms,built-up area,furnishing,age,floor,total floors,monthly rent
0,other society,Residential Apartment,Bandra (West),2,800.0,Furnished,5 to 10 years old,2,6,85000
1,other society,Residential Apartment,Khar West,2,1050.0,Furnished,1 to 5 years old,8,12,110000
2,Gajra Bhoomi Symphony,Residential Apartment,Sector-20 Koparkhairane,2,1050.0,Unfurnished,1 to 5 years old,14,17,32000


In [62]:
from sklearn.preprocessing import LabelEncoder

society_encoder = LabelEncoder().fit(df1['society'])
df1['society'] = society_encoder.transform(df1['society'])

In [65]:
society_encoder.classes_

array(['Aaram Sty', 'Acme Ozone', 'Adityavardhan Apartment',
       'Ajmera Aeon', 'Ajmera Bhakti Park Sector 1 and 2', 'Ajmera Zeon',
       'Alica Nagar CHS', 'Ashford Royale', 'Ashwini CHS',
       'Bhagwati Bay Bliss', 'Bholenath Chembur Castle', 'Bhoomi Park',
       'Carter Road', 'Central Avenue Apartments', 'Chaitanya Towers',
       'Chatrapati Shivaji Raje Complex', 'Check', 'Co-oprative Society',
       'Cosmos Jewels', 'Cosmos Lounge', 'Crescent Bay', 'DB Woods',
       'Description :', 'Dheeraj Platinum Apartment', 'Diamond Garden',
       'Divine Varsha', 'Dosti Acres', 'Dosti Ambrosia', 'Dosti Estates',
       'Dosti Imperia', 'Dynamix Parkwoods', 'Emgee Greens',
       'Evening Glory CHS', 'Evershine Millennium Paradise',
       'Exquisit Falt', 'Exquisite Falt', 'Gajra Bhoomi Symphony',
       'Geomatrix Silver Crest', 'Godrej Central', 'Godrej Serenity',
       'Grace Heritage', 'Grit Heights CHS', 'Gundecha Altura',
       'HDIL Dreams', 'HDIL Metropolis Tower',
    

In [66]:
type_encoder = LabelEncoder().fit(df1['type'])
df1['type'] = type_encoder.transform(df1['type'])

In [67]:
type_encoder.classes_

array(['Independent House/Villa', 'Residential Apartment',
       'Studio Apartment'], dtype=object)

In [68]:
furnishing_encoder = LabelEncoder().fit(df1['furnishing'])
df1['furnishing'] = furnishing_encoder.transform(df1['furnishing'])

In [69]:
furnishing_encoder.classes_

array(['Furnished', 'Not Mentioned', 'Semifurnished', 'Unfurnished'],
      dtype=object)

In [70]:
location_encoder = LabelEncoder().fit(df1['location'])
df1['location'] = location_encoder.transform(df1['location'])

In [71]:
location_encoder.classes_

array(['4 Bunglows', '7 Bunglow', 'Airoli', 'Ambedkar Nagar',
       'Anand Nagar', 'Andheri (East)', 'Andheri (West)', 'Asha Nagar',
       'Bandra (West)', 'Bhakti Park', 'Bhandup (West)',
       'Borivali (East)', 'Borivali (West)', 'Chakala', 'Chandivali',
       'Chembur', 'Chembur (East)', 'Chembur (West)', 'Chincholi Bunder',
       'Colaba', 'Cuffe Parade', 'Dadar (East)', 'Dahisar (East)',
       'Dahisar (West)', 'Deonar', 'Dhokali', 'Dina Bama Estate',
       'Dindoshi', 'Dn Nagar', 'Dombivli (East)', 'Ekta Nagar',
       'Evershine Nagar', 'Ganesh Nagar', 'Gavanpada', 'Ghansoli',
       'Ghatkopar West', 'Ghodbunder Road', 'Gokuldham',
       'Goregaon (East)', 'Goregaon (West)', 'Hill Road',
       'Hiranandani Estate', 'Hiranandani Gardens - Powai',
       'Hiranandani Meadows', 'Hmpl Surya Nagar', 'I C Colony',
       'Jacob Circle', 'Jankalyan Nagar', 'Jogeshwari (East)', 'Juhu',
       'Jvlr', 'Jvpd', 'Kalina', 'Kalyan (West)', 'Kamothe', 'Kandarpada',
       'Kandival

In [72]:
age_encoder = LabelEncoder().fit(df1['age'])
df1['age'] = age_encoder.transform(df1['age'])

In [73]:
age_encoder.classes_

array(['0 to 1 years old', '1 to 5 years old', '10+ years old',
       '5 to 10 years old', 'Not Mentioned'], dtype=object)

In [74]:
df1

,society,type,location,bedrooms,built-up area,furnishing,age,floor,total floors,monthly rent
0,176,1,8,2,800.0,0,3,2,6,85000
1,176,1,63,2,1050.0,0,1,8,12,110000
2,36,1,128,2,1050.0,3,1,14,17,32000
3,20,1,88,3,2275.0,2,1,40,41,150000
4,8,1,150,1,550.0,2,3,4,7,27000
...,...,...,...,...,...,...,...,...,...,...
12792,176,1,94,3,2529.0,0,1,12,32,175000
12793,150,1,83,3,1382.0,2,0,27,40,48000
12794,176,1,165,3,1750.0,3,2,9,16,200000
12795,176,1,109,2,950.0,2,3,8,10,25500


##  Decision Tree Regressor model

In [75]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df1.drop('monthly rent', axis = 1), df1['monthly rent'], test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10237, 9), (2560, 9), (10237,), (2560,))

In [78]:
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor()
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [79]:
r2_score(y_test, dt_model.predict(X_test))

0.6464749701532908

In [80]:
Final_Scores['Decision Tree Regressor %'] = [r2_score(y_train, dt_model.predict(X_train))*100,
                                             r2_score(y_test, dt_model.predict(X_test))*100]

## Random Forest Regressor model

In [81]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [82]:
r2_score(y_test, rf_model.predict(X_test))

0.8405331288973058

In [83]:
Final_Scores['Random Forest Regressor %'] = [r2_score(y_train, rf_model.predict(X_train))*100,
                                             r2_score(y_test, rf_model.predict(X_test))*100]

#Final_Scores 

In [84]:
Final_Scores

,Linear Regression %,Ridge Regression %,Lasso Regression %,Decision Tree Regressor %,Random Forest Regressor %
With Training Data,87.516062,87.515960,87.512821,99.945316,97.495486
With Test Data,88.502509,88.504798,88.510271,64.647497,84.053313
